# Pokročilé dolování z databází - Průběžná zpráva
### Autoři: Lukáš Vala, Dan Balarin

# Použitá data
V tomto návodu budeme pracovat s dvěma datasety:
 - [Adult](https://archive-beta.ics.uci.edu/dataset/2/adult) - Obsahuje data z roku 1994. Obsahuje informace o lidech z celého světa a k tomu navíc, zda daná osoba vydělává více či méně než 50000 dolarů za rok. Můžeme si tedy pokládat otázky, jak ovlivňuje plat národnost, rasa, pohlaví a podobně.
    - Počet řádků: 48842
    - Počet slouců: 14
 - [GDP in 1994](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?end=1994&most_recent_year_desc=true&start=1993) - Obsahuje informace o hrubém domácím produktu jednotlivých států v roce 1994. 
    - Počet řádků: 266
    - Počet slouců: 3 - jeden z těchto sloupců bude v tomto návodu smazán

 Oba datasety obsahují sloupec stát - proto je budeme dále v dokumentu spojovat podle tohoto sloupce.

# Vypracování návodu pro zpracování a analýzu dat pomocí systému CleverMiner
## Krok 0 Instalace potřebných knihoven
- Clever Miner: `pip install cleverminer`
- pandas: `pip install pandas`
## Krok 1 Načtení a zpracování dat
Prvním krokem je načíst data pomocí knihovny pandas. Data se v tomto případě nacházejí ve složce data:

In [ ]:
import pandas as pd

adult = pd.read_csv ('./data/adult.data', encoding='cp1250', sep=', ')
gdp = pd.read_csv ('./data/gdpStates.txt', encoding='cp1250', sep='\t')

adult.head(5)
gdp.head(5)

Pomocí parametru `sep` je důležité specifikovat, jaký separátor jednotlivých atributů je v tabulce použit. U datasetu adult je to `, ` a u GDP je to tabulátor. Správnost načtení dat můžeme poté zkontrolovat pomocí výpisu prvních 5 hodnot obou tabulek.

Dále bychom si také měli zkontrolovat, zda pandas správně určila datové typy u jednotlivých sloupců. 
To můžeme zjistit pomocí příkazu `dtypes`:

In [22]:
print(adult.dtypes)
print(gdp.dtypes)

Age                int64
Workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
country           object
income            object
dtype: object
country              object
Most Recent Year      int64
GDP per Capita      float64
dtype: object


Pro string vypíše pandas `object`, pro integer `int64` a pro desetinné číslo `float64`

## Krok 2 - Spojení dat
Nyní je třeba spojit 2 tabulky do sebe, abychom mohli analýzu provádět nad datasetem Adult společně s informací o hrubém domácím produktu státu, v němž byl daný člověk narozen. 

Oba datasety obsahují sloupec country. V tabulce `adult` se tento sloupec jmenuje `native-country`, je proto nutné ho přejmenovat přímo v datasetu na `country`, aby se shodoval s názvem sloupce ve druhém datasetu. Dále je potřeba upravit jednotlivé hodnoty v jedné z tabulek, protože např. hodnota `United States` v tabulce adult je původně zapsána v gdp tabulce jako `United-States`. Použijeme tedy funkci definovanou v ukázce pod tímto textem a následně v datech GDP najdeme jednotlivé hodnoty a nahradíme je správnými hodnotami z tabulky adult (použita funkce najít a nahradit ve VS-code :) ).

In [23]:
def getUniqueValuesFromColumn(df, col):
    values = df[col].tolist()
    uniqueValues = []
    for i in values:
        if(i not in uniqueValues):
            uniqueValues.append(i)
    return uniqueValues

uniqueCountryValues = getUniqueValuesFromColumn(adult, "country")

Před samotným spojením tabulek ještě odstraníme z GDP tabulky ty státy, které nejsou obsaženy v tabulce adult, aby po spojení nedošlo k vytvoření několika řádků s prázdnými hodnotami (vyplněný by byl pouze stát). Použijeme proto další funkci, do které přidáme dataset, název sloupce a pole hodnot států. Funkce nám pak vráti dataset obsahující pouze řádky se státem, který byl obsažený v poskytnutém poli (uniqueCountryValues z části před touto).

In [24]:
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]

gdp = filter_rows_by_values(gdp, "country", uniqueCountryValues)

Poté již spojíme datasety a vznikne nám nový, který obsahuje všechny atributy z datasetu adult a navíc hrubý domácí produkt státu, ze kterého daná osoba pochází:

In [25]:
adultWithGdp = pd.merge(adult, gdp, on='country', how='outer')

Nyní můžeme vytvořit nové atributy v datasetu podle našich potřeb. Následující kód vytvoří nový sloupec `AgeStatus` a připíše hodnotu "Young" lidem mladším než 30, "Middle aged" lidem mezi 30 a 60 a "Old" lidem starším než 60.

In [26]:
adultWithGdp['AgeStatus'] = adultWithGdp.apply(lambda row: "Young" if row.Age < 30 else ("Middle aged" if row.Age >= 30 & row.Age < 60 else "Old"), axis=1)

## Krok 3 - <a href="https://lispminer.vse.cz/wiki/doku.php?id=mft:start">4ft miner</a>

Reference na [Clever Miner](https://www.cleverminer.org/docs-page.html#section-4)

Analytická procedura 4ft-Miner hledá všechny zajímavé frekvence kombinací hodnot kategorií dvou předzpracovaných atributů, navíc počítané na podmnožinách analyzovaných dat zadaných pomocí booleovských atributů s bohatou syntaxí.
 
U 4ft mineru nastavuje základní parametry:
`conf`: Konfidence - pravděpodobnost, že podmínka bude vyplněna
`base`: Základ - počet vyhovujících případů
`minlen, maxlen`: Maximální a minimální délka podmnožin/sekvencí/cutů

### Příklad 1

V Clever Mineru používáme 4ft miner následovně:

In [ ]:
from cleverminer import cleverminer

clm = cleverminer(df=adultWithGdp, proc='4ftMiner',
               quantifiers= {'conf':0.7, 'Base':18000},
               ante ={
                    'attributes':[
                        {'name': 'AgeStatus', 'type': 'subset', 'minlen': 1, 'maxlen': 2}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'},
               succ ={
                    'attributes':[
                        {'name': 'income', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'}
                )

V předchozí ukázce jako antecedent nastavujeme vytvořený atribut `AgeStatus` s minimální délkou atributu 1 a maximální délkou 2. To znamená, že clever miner bude vyhledávat podmnožiny z atributu AgeStatus o délce 1 - 2. Jako výsledek tedy můžeme dostat například `AgeStatus(Middle aged)`(délka podmnožiny 1), nebo např. `AgeStatus(Middle aged, Old)`(délka podmnožiny 2), což nám říká, že Age status daného člověka je buď Middle aged nebo Old. Sukcedent jsme nastavili na `income`. Vzhledem k tomu, že atribut `income` může nabývat pouze dvou hodnot, nedává v tomto případě smysl nastavovat maximální délku podmnožiny na 2 - byly by obsaženy všechny hodnoty z `income`.

Konfidenci jsme nastavili na 70% a základ 18 000.

Přepisem do lidsky srozumitelného jazyka toto znamená: Jestliže si vezmeme pro osobu jednu nebo dvě hodnoty z `AgeStatus`(Old/Yound/Middle aged), je v nějakém případě pravděpodobnost, že daná osoba bude vydělávat více či méně než 50K, vyšší než 70% a zároveň je takových osob v datasetu více než 18 000?

Výsledek vypíšeme následovně:

Pomocí následujícího příkazu můžeme zjistit, zda Clever Miner takové pravidlo v datech našel:

In [28]:
clm.print_rulelist()


List of rules:
RULEID BASE  CONF  AAD    Rule
     1 24720 0.759 +0.000 AgeStatus(Middle aged Young) => income(<=50K) | ---



Z výpisu vydíme, že jedno takové pravidlo existuje. Detaily o něm vypíšeme následovně:

In [29]:
clm.print_rule(1)



Rule id : 1

Base : 24720  Relative base : 0.759  CONF : 0.759  AAD : +0.000  BAD : +0.000

Cedents:
  antecedent : AgeStatus(Middle aged Young)
  succcedent : income(<=50K)
  condition  : ---

Fourfold table
    |  S  |  ¬S |
----|-----|-----|
 A  |24720| 7841|
----|-----|-----|
¬A  |    0|    0|
----|-----|-----|



Z tabulky vidíme, že Clever Miner skutečně našel takový vztah mezi `AgeStatus` a `income`. Konkrétně z výsledku můžeme vyvodit toto pravidlo: jestliže je někdo `Middle aged` (30-60), nebo `Young` (< 30), pak je šance 75.9%, že bude mít plat menší nebo rovno 50 tisíc dolarů za rok a zároveň takových lidí je 24 720 v celém datasetu.

### Příklad 2
Zajímavý výsledek dostaneme, pokud sledujeme korelaci atributů `marital-status` a `income`: 

In [ ]:
clm = cleverminer(df=adultWithGdp, proc='4ftMiner',
    quantifiers= {'conf':0.95, 'Base':10000},
    ante ={
        'attributes':[
            {'name': 'marital-status', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
        ], 'minlen':1, 'maxlen':1, 'type':'con'},
    succ ={
        'attributes':[
            {'name': 'income', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
        ], 'minlen':1, 'maxlen':1, 'type':'con'}
)

Jak si z kódu můžeme povšimnout, snažíme se nyní najít vztah mezi manželským statusem a příjmem s velmi vysokou konfidencí - 95%. Otázka tedy nyní zní: Pokud si vezmeme člověka s určitým manželským statusem, je v nějakém případě pravděpodobnost, že daná osoba vydělává více či méně než 50k za rok, větší než 95% a zároveň takových osob je více než 10 000?

Po spuštění zjistíme, že takové pravidlo skutečně existuje:

In [31]:
clm.print_rulelist()


List of rules:
RULEID BASE  CONF  AAD    Rule
     1 10192 0.954 +0.257 marital-status(Never-married) => income(<=50K) | ---



Podíváme se tedy na detaily:

In [32]:
clm.print_rule(1)



Rule id : 1

Base : 10192  Relative base : 0.313  CONF : 0.954  AAD : +0.257  BAD : -0.257

Cedents:
  antecedent : marital-status(Never-married)
  succcedent : income(<=50K)
  condition  : ---

Fourfold table
    |  S  |  ¬S |
----|-----|-----|
 A  |10192|  491|
----|-----|-----|
¬A  |14528| 7350|
----|-----|-----|



Z výsledku můžeme vyvodit následující závěr: Jestliže osoba nikdy nebyla v manželském vztahu, pak je 95% šance, že vydělává méně než 50k za rok a zároveň je takových lidí více než 10000. 

Z tabulky si ale můžeme také povšimnout dalšího pravidla: Pokud si osoba v životě již někoho vzala, pak je 33,5% šance, že má plat vyšší než 50k za rok - vypočteno na kalkulačce jako 7350(počet lidí, kteří si již někoho vzali a vydělávají více než 50k za rok) / (14528 + 7350(Součet všech lidí, kteří si již někoho vzali)). Teoreticky tedy můžeme říci, že začátek manželského vztahu zvyšuje pravděpodobnost, že daná osoba vydělává více, než 50k za rok, o 28.5% -> více než šestinásobně.

V kódu je uvedeno několik dalších příkladů na 4ft miner s popisem v komentářích.

## Krok 4 - [CF Miner](https://lispminer.vse.cz/guhate/doku.php?id=lm_guha_te_cf_proc)

Reference na [Clever Miner](https://www.cleverminer.org/docs-page.html#section-4)

Analytická procedura CF-Miner hledá všechna zajímavá rozdělení frekvencí kategorií předzpracovaného atributu počítaná na podmnožinách analyzovaných dat zadaných pomocí [booleovských atributů s bohatou syntaxí](https://lispminer.vse.cz/wiki/doku.php?id=lmtask:settings:ftcedent).

Analytickou úlohu v CF mineru můžeme definovat například takto:

Existuje velmi výrazně převyšující hodnota atributu `income` pro nějakou kombinaci údajů `marital-status` a `relationship`?

Vzhledem k tomu, že nás zajímá výrazně převažující hodnta, nastavíme kvantifikátor `RelMax` na 0.99. CF miner pak bude hledat kombinaci hodnot atributů `marital-status` a `relationship` tak, aby jedna z hodnot byla obsažena minimálně v 99% případů. Pomocí kvantifikátoru `Base` specifikujeme počet záznamů vyhovující podmínkám. Reference na další kvantifikátory lze nalézt v dokumentaci [Clever mineru](https://www.cleverminer.org/docs-page.html#section-4) v sekci CF miner.

Ve slovníku cond si dále můžeme povšimnout dvou již zmíněných atributů. Syntax je zde stejný jako u 4ft-mineru. Povšimněme si ale nyní definice parametrů `minlen` a `maxlen` za polem atributů. `maxlen` je v tomto případě třeba nastavit na 2, aby vyhledával CF miner vztah mezi `income` a <b>kombinací</b> atributů `marital-status` a `relationship`.

In [ ]:
clm = cleverminer(df=adultWithGdp,target='income',proc='CFMiner',
    quantifiers= {'Base': 4000, 'RelMax': 0.99},
    cond = {
        'attributes':[
            {'name': 'marital-status', 'type': 'subset', 'minlen': 1, 'maxlen': 1},
            {'name': 'relationship', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
        ], 'minlen':1, 'maxlen':2, 'type':'con'}
)

Po spuštění tohoto kódu vypíšeme pravidlo, které CF miner našel:

In [34]:
clm.print_rule(1)



Rule id : 1

Base :  4485  Relative base : 0.138  Steps UP (consecutive) :     0  Steps DOWN (consecutive) :     1  Steps UP (any) :     0  Steps DOWN (any) :     1  Histogram maximum :  4451  Histogram minimum :    34  Histogram relative maximum : 0.992 Histogram relative minimum : 0.008

Condition  : marital-status(Never-married) & relationship(Own-child)

Histogram [4451, 34]



Vidíme, že CF miner skutečně takový vztah našel, dokážeme vyčíst, že pokud má člověk dítě a nikdy nebyl v manželství, pak má 99% šanci, že vydělává méně než 50k za rok a zároveň takových lidí je v datasetu minimálně 4400. V histogramu vidíme: [4451, 34] - pouze 34 takových lidí z celkových 4485 vydělává více než 50k za rok.

Více příkladů s CF-minerem sem bude přidáno. Některé se již nachází v kódu.